## Download the Data

In [ ]:
import os
import pandas as pd
import tarfile
import urllib.request
import matplotlib.pyplot as plt
import numpy as np

DOWNLOAD_URL = "https://github.com/fatimaezzahra-creator/Projet-ML/raw/refs/heads/main/datasets/adult.tgz"
DATASET_PATH = "datasets"

def fetch_data(data_url, data_path):
    if not os.path.isdir(data_path):
        os.makedirs(data_path)
    tgz_path = os.path.join(data_path, "adult.tgz")
    urllib.request.urlretrieve(data_url, tgz_path)
    tgz_file = tarfile.open(tgz_path)
    tgz_file.extractall(path=data_path)
    tgz_file.close()    

fetch_data(DOWNLOAD_URL, DATASET_PATH)

## Dataset Exploratory Analysis

### Analysis of Form

In [ ]:
#load the data
def load_data():
    csv_path = os.path.join(DATASET_PATH, "adult.data")
    return pd.read_csv(csv_path)

data = load_data()
data.info()

In [ ]:
import seaborn as sns
#Missing Data Visualization
sns.heatmap(data.isna(), cbar=False)

We can see that the graph is all dark, which means there is no missing values in the data .

### Analysis of Content

In [ ]:
#Target Distribution Analysis
data[" class"].value_counts(normalize=True)

In [ ]:
import seaborn as sns
# Numerical attribute
for col in data.select_dtypes("int64"):
    sns.displot(data[col])

In [ ]:
#Categorical attribute
import matplotlib.pyplot as plt
for col in data.select_dtypes("object"):
    sns.displot(data=data, x=col)
    plt.title(f" '{col}'", fontsize=16)
    plt.xlabel(col, fontsize=12)
    plt.ylabel("Nombre d'observations", fontsize=12)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
#copy of the data
df=data.copy()
# Supprimer les espaces des noms de colonnes et des valeurs
df.columns = df.columns.str.strip()
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
# Creating subsets based on the target variable
class_0 = df[df["class"] == "<=50K"]
class_1 = df[df["class"] == ">50K"]
combined_df = (pd.concat([class_0, class_1]))

In [ ]:
#relation target-age
import seaborn as sns
sns.histplot(
  data=combined_df ,
  x="age",
  hue="class",
  stat="density",
  common_norm=False,
  palette="muted");

In [ ]:
#relation target-education_num
sns.histplot(
  data=combined_df ,
  x="education-num",
  hue="class",
  stat="density",
  common_norm=False,
  palette="muted");

In [ ]:
#ralation target-education
sns.histplot(
  data=combined_df ,
  x="education",
  hue="class",
  stat="density",
  common_norm=False,
  palette="muted");
plt.xticks(rotation=90)
plt.show()

**The features education (categorical) and education-num (numerical) may convey similar information, as they both represent the education level of an individual.**

In [ ]:
education_mapping = data.groupby(" education")[" education-num"].unique()
# Afficher le mapping pour vérifier la correspondance
for edu, edu_num in education_mapping.items():
    print(f"Education: {edu}, Education_Num: {edu_num}")

The output of this mapping shows that each education category corresponds to a single unique value of education-num. This confirms that the two features are effectively encoding the same information.

In [ ]:
#relation target-workclass
sns.histplot(
  data=combined_df ,
  x="workclass",
  hue="class",
  stat="density",
  common_norm=False,
  palette="muted");
plt.xticks(rotation=90)
plt.show()

In [ ]:
#relation target-sex
sns.histplot(
  data=combined_df ,
  x="sex",
  hue="class",
  stat="density",
  common_norm=False,
  palette="muted");

In [ ]:
#relation target-marital status
sns.histplot(
  data=combined_df ,
  x="marital-status",
  hue="class",
  stat="density",
  common_norm=False,
  palette="muted");
plt.xticks(rotation=90)
plt.show()

In [ ]:
#relation target-relationship
sns.histplot(
  data=combined_df ,
  x="relationship",
  hue="class",
  stat="density",
  common_norm=False,
  palette="muted");

**The features relationship and marital-status  might convey similar information because a person's relationship type often depends on their marital status.**



In [ ]:
education_mapping = data.groupby(" marital-status")[" relationship"].unique()

# Afficher le mapping pour vérifier la correspondance
for mrs, rshp in education_mapping.items():
    print(f"marital-status: {mrs}, relationship: {rshp}")

The output reveals that for each value of marital-status, there are multiple possible values for relationship.
This variability indicates that a person's relationship cannot be uniquely determined based on their marital-status.

In [ ]:
#relation taget-fnlwgt
sns.histplot(
  data=combined_df ,
  x="fnlwgt",
  hue="class",
  stat="density",
  common_norm=False,
  palette="muted");

In [ ]:
# relation target-race
sns.histplot(
  data=combined_df ,
  x="race",
  hue="class",
  stat="density",
  common_norm=False,
  palette="muted");


In [ ]:
#relation target-native country
sns.histplot(
  data=combined_df ,
  x="native-country",
  hue="class",
  stat="density",
  common_norm=False,
  palette="muted");
plt.xticks(rotation=90)
plt.show()

The native-country  and race columns in the dataset contains many unique values, some of which have very low frequencies. Keeping all these rare categories can negatively impact the machine learning model due to:

Overfitting: The model may place undue importance on rare categories, learning patterns that don't generalize well to new data.
Increased Complexity: High cardinality increases the dimensionality during encoding (e.g., in one-hot encoding), which can slow down training and complicate the model unnecessarily.


In [ ]:
#relation target-occupation
sns.histplot(
  data=combined_df ,
  x="occupation",
  hue="class",
  stat="density",
  common_norm=False,
  palette="muted");
plt.xticks(rotation=90)
plt.show()

In [ ]:
#rlation target-capital gain
sns.histplot(
  data=combined_df ,
  x="capital-gain",
  hue="class",
  stat="density",
  common_norm=False,
  palette="muted");

In [ ]:
#ralation target-capitalloss
sns.histplot(
  data=combined_df ,
  x="capital-loss",
  hue="class",
  stat="density",
  common_norm=False,
  palette="muted");

**The combination of capital-gain and capital-loss into a single derived feature could have a stronger correlation with the target variable (class) than either capital-gain or capital-loss individually, potentially improving the predictive power of the model.**

In [ ]:
#Capital Features Combination
dff=df.copy()
combined_df["capital_features"] = dff["capital-gain"] - combined_df["capital-loss"]
#Capital Features Combination
dff=df.copy()
from sklearn.preprocessing import LabelEncoder
#add the column of capital_feature to the data
dff["capital_net"]=(dff["capital-gain"] -dff["capital-loss"])
dff["ratio"]=(dff["capital-gain"] /(dff["capital-loss"]+0.00000001))
dff["capital_weighted"]=(dff["capital-gain"]*0.223329 +dff["capital-loss"]*0.150526)


# Encodage LabelEncoder pour chaque colonne catégorique
label_encoders = {}
for col in dff.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    dff[col] = le.fit_transform(dff[col])
    label_encoders[col] = le  # Stocker l'encodeur pour chaque colonne (optionnel, utile pour l'inverse_transform)

# Calcul de la matrice de corrélation
corr_matrix = dff.corr()
corr_matrix["class"].sort_values(ascending=False)

**capital_net: the net difference between gains and losses**

Correlation: 0.214, lower than capital-gain. Relevance: While intuitive (netting gains and losses), this feature does not add much value compared to capital-gain alone. Consider dropping it unless it improves model performance.

**ratio: relative proportion of gains to losses**

Correlation: 0.223, identical to capital-gain. Relevance: This feature does not improve upon capital-gain’s correlation. Its usefulness might depend on the model's capacity to interpret non-linear relationships, but it seems redundant for linear models.

**capital_weighted: weighted sum of the two based on their importance**

Correlation: 0.229, slightly higher than capital-gain (0.223). This feature combines the effects of both gains and losses, weighted by their individual correlations with class. It shows a slight improvement, suggesting it may capture some additional nuanced information. This feature is pertinent to keep for modeling.

In [ ]:
#relation target- hours per week
sns.histplot(
  data=combined_df ,
  x="hours-per-week",
  hue="class",
  stat="density",
  common_norm=False,
  palette="bright");

**synthesis**

Retain education-num (numerical feature) and remove education to avoid redundancy and simplify the dataset.

Retain both features relationship and marital-status as they capture different aspects of an individual's social situation.

Focus on the most impactful features:
education-num, age, hours-per-week, capital_weighted, and categorical variables such as relationship and marital-status.

fnlwgt  adds minimal value to the predictive power of the model.

Reduce noise, we group all rare categories (those with a frequency below a certain threshold, e.g., 50 occurrences) into a single category called "Other".

## Data Pre-Processing


### Data Cleaning Process

**Removing Low-Impact Features** 

In [ ]:
# Combine capital features in capital_weighted
df["capital_weighted"] = df["capital-gain"] * 0.223329 + df["capital-loss"] * 0.150526

# Delete the redundant columns
df.drop(["education", "fnlwgt", "capital-gain", "capital-loss"], axis=1, inplace=True)


**Grouping Rare Categories into 'Other' to Simplify Data**

In [ ]:
# Count the frequency of each category 
native_country_counts = df['native-country'].value_counts()
race_counts = df['race'].value_counts()

# Identify categories to keep 
to_keep = native_country_counts[native_country_counts >= 500].index
to_keep_race = race_counts[race_counts >= 500].index

# Replace rare categories with "Other"
df['native-country'] = df['native-country'].apply(lambda x: x if x in to_keep else 'Other')
df['race'] = df['race'].apply(lambda x: x if x in to_keep_race else 'Other')

# Verify the updated frequencies
print(df['native-country'].value_counts())
print(df['race'].value_counts())

### Encoding and Scaling

In [ ]:
# Transformation of Text and Categorical Data
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

numerical_features = df.select_dtypes(include=np.number).columns.tolist()
categorical_features = df.select_dtypes(include=['object']).columns.tolist()

preprocessor = ColumnTransformer([
    ("categorical", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ("numerical", StandardScaler(), numerical_features)
])

### Train/Test Split

We saw earlier that the distribution of the target class is NOT balanced, so to create our train and test sets we can use a StratifiedShuffleSplit that will not only shuffle the instances but also preserve the proportions in the original dataset.

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

target_name = 'class'
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_indexes, test_indexes in split.split(df, df[target_name]):
    train_set = df.iloc[train_indexes]
    test_set = df.iloc[test_indexes]

# Supprimer les espaces des noms de colonnes et des valeurs
train_set.columns = train_set.columns.str.strip()
train_set = train_set.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Supprimer les espaces des noms de colonnes et des valeurs
test_set.columns = test_set.columns.str.strip()
test_set = test_set.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

print("Proportions in the original dataset:", df[target_name].value_counts(normalize=True))
print("Proportions in the train set:", train_set[target_name].value_counts(normalize=True))
print("Proportions in the test set:", test_set[target_name].value_counts(normalize=True))

### Separate target from the features

In [ ]:
train_data = train_set.drop(target_name, axis=1)
target = train_set[target_name].copy()

## Model Selection
In this section, we will train 5 different models, evaluate them and compare to find the best one. They are:
* A LogisticRegression;
* An SGDClassifier;
* A RandomForestClassifier;
* A GradientBoostingClassifier;
* A KNeighboursClassifier;

First let's build each pipeline, with our previously defined preprocessor.

In [ ]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

LR_clf = LogisticRegression(solver='liblinear', random_state=42)
SGD_clf = SGDClassifier(loss='hinge', random_state=42)
RF_clf = RandomForestClassifier(random_state=42)
GB_clf = GradientBoostingClassifier(random_state=42)
KNN_clf = KNeighborsClassifier()

LR_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LR_clf)
])
SGD_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SGD_clf)
])
RF_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RF_clf)
])
GB_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', GB_clf)
])
KNN_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', KNN_clf)
])

models = {
    'Logistic': LR_pipeline,
    'SGD': SGD_pipeline,
    'RandomForet': RF_pipeline,
    'GradientBoosting': GB_pipeline,
    'KNeighbours': KNN_pipeline 
}

Now we can define the parameter grid for each model.

In [ ]:
LR_param_grid = {
    'classifier__penalty': ['l1','l2'],
    'classifier__C': [0.01, 0.1, 1],
    'classifier__max_iter' : [100, 500, 1000]
}

SGD_param_grid = {
        'classifier__learning_rate': ['constant', 'invscaling'],
        'classifier__eta0': [0.01, 0.1, 1],
        'classifier__penalty': ['l2', 'l1'],
        'classifier__alpha': [0.001, 0.01, 0.1],
        'classifier__max_iter': [100, 500, 1000]
}

RF_param_grid = {
    'classifier__n_estimators': [10, 50, 100],
    'classifier__max_depth': [10, 50, 100]
}

GB_param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'classifier__max_depth': [3, 4, 5, 6],
    'classifier__subsample': [0.7, 0.8, 0.9, 1.0]
}

KNN_param_grid = {
    'classifier__n_neighbors': [3, 5, 7, 9, 11, 15],
    'classifier__weights': ['uniform', 'distance'],
    'classifier__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'classifier__leaf_size': [10, 20, 30, 40, 50],
    'classifier__p': [1, 2] 
}

param_grid = {
    'Logistic': LR_param_grid,
    'SGD': SGD_param_grid,
    'RandomForet': RF_param_grid,
    'GradientBoosting': GB_param_grid,
    'KNeighbours': KNN_param_grid 
}

Finally, we perform a GridSearchCV. Since the parameter grid are already pretty extensive, we will use only 3 folds.

In [ ]:
from sklearn.model_selection import GridSearchCV

for name, model in models.items():
    print(f"For the {name} model:")
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid[name],
        cv=3,                   # Validação cruzada 5-fold
        scoring='accuracy',     # Métrica de avaliação
        n_jobs=-1,              # Uso de todos os núcleos disponíveis
    )
    grid_search.fit(train_data, target)
    print("    Best Hyperparams:", grid_search.best_params_)
    print("    Best Accuracy:", grid_search.best_score_)